<a href="https://colab.research.google.com/github/delibae/health_care_pr/blob/main/project_2/%EC%9D%B4%EA%B2%B8%ED%9D%AC/%EA%B3%BC%EC%A0%9C2_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <header1>과제2</header1>

### 이겸희_성균관대 

### 학습 준비단계

In [2]:
from google.colab import drive

In [3]:
drive.mount('/content/drive')
print('g-drive mounted.')

Mounted at /content/drive
g-drive mounted.


In [ ]:
!unzip -q -o /content/drive/MyDrive/TL_81_단일.zip

In [12]:
!unzip -q -o /content/drive/MyDrive/TS_81_단일.zip

In [65]:
import pandas as pd
import numpy as np
import sys
import os
from matplotlib import colors
import random
from pathlib import Path
import imageio
import skimage
import skimage.io
import skimage.transform
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import scipy
from keras.models import load_model
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras import optimizers
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout, BatchNormalization,LeakyReLU
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from keras.utils import to_categorical
import cv2


print("NumPy Version :{}".format(np.__version__))
print("TensorFlow Version :{}".format(tf.__version__))
print("Matplotlib Version :{}".format(plt.matplotlib.__version__))

NumPy Version :1.21.6
TensorFlow Version :2.9.2
Matplotlib Version :3.2.2


In [66]:
IMAGE_PATH = "/content/PNG/"
IMAGE_WIDTH = 64
IMAGE_HEIGHT = 64
IMAGE_CHANNELS = 3
IMG_SIZE = 64
RANDOM_STATE = 2018
TEST_SIZE = 0.2
VAL_SIZE = 0.2
MAX_POOL_DIM = 2
KERNEL_SIZE = 3
PATIENCE = 5
VERBOSE = 1

In [67]:
plt.rc("font", family='Malgun Gothic')
plt.rc('axes', unicode_minus = False)

In [68]:
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', 'This pattern has match groups')
warnings.filterwarnings('ignore', 'The iterable function was deprecated in Matplotlib')

### 데이터 가져오기

In [95]:
png_cls = sorted(os.listdir("/content/PNG"))
json_cls = sorted(os.listdir("/content/json_folder"))

print((json_cls))

['K-038884_json', 'K-038890_json', 'K-038896_json', 'K-038910_json', 'K-038912_json', 'K-038913_json', 'K-038914_json', 'K-038927_json', 'K-038929_json', 'K-038954_json', 'K-038958_json', 'K-038959_json', 'K-038962_json', 'K-038967_json', 'K-038970_json', 'K-039021_json', 'K-039036_json', 'K-039047_json', 'K-039104_json', 'K-039108_json', 'K-039123_json', 'K-039136_json', 'K-039146_json', 'K-039147_json']


In [96]:
ts_list = []
tl_list = []

for file in png_cls:
        tl_list.append(os.listdir("/content/PNG" +'/' + file))
for file in json_cls:
        ts_list.append(os.listdir("/content/json_folder" +'/' + file))
ts_list = np.concatenate(ts_list).tolist()
tl_list = np.concatenate(tl_list).tolist()

In [97]:
df = pd.DataFrame({
    'PNG' : ts_list,
    'JSON' : tl_list
})
df

,PNG,JSON
0,K-038884_0_0_1_1_90_080_200.json,K-038884_0_0_0_0_90_180_200.png
1,K-038884_0_0_0_0_90_040_200.json,K-038884_0_0_1_0_90_020_200.png
2,K-038884_0_0_0_1_75_100_200.json,K-038884_0_0_1_2_90_060_200.png
3,K-038884_0_0_1_1_90_140_200.json,K-038884_0_0_0_2_90_200_200.png
4,K-038884_0_0_1_1_75_160_200.json,K-038884_0_0_1_2_90_300_200.png
...,...,...
9607,K-039147_0_0_0_1_70_260_200.json,K-039147_0_0_0_1_70_200_200.png
9608,K-039147_0_1_0_2_75_120_200.json,K-039147_0_0_0_1_90_200_200.png
9609,K-039147_0_1_1_1_90_260_200.json,K-039147_0_1_1_0_70_300_200.png
9610,K-039147_0_0_1_0_60_340_200.json,K-039147_0_1_0_1_60_320_200.png


In [98]:
import json
import os
from tqdm import tqdm

In [99]:
path = "/content/json_folder/" 

In [100]:
dl_name = []
PNG_name = []
for file, file2 in zip(json_cls,png_cls):
    file_list = []
    file_list.append(os.listdir(path + file))
    file_list = np.concatenate(file_list).tolist()
    for j_file in file_list:
        with open(path + file + '/' + j_file, "r", encoding='utf-8') as f:
            contents = f.read()
            json_data = json.loads(contents)
            dl_name.append(json_data['images'][0]['dl_name'])
            PNG_name.append(file2 + '/' + json_data['images'][0]['imgfile'])

In [101]:
df2 = pd.DataFrame({
    'PNG' : PNG_name,
    'Medicine_Name' : dl_name
})
df2

,PNG,Medicine_Name
0,K-038884/K-038884_0_0_1_1_90_080_200.png,에피나레정
1,K-038884/K-038884_0_0_0_0_90_040_200.png,에피나레정
2,K-038884/K-038884_0_0_0_1_75_100_200.png,에피나레정
3,K-038884/K-038884_0_0_1_1_90_140_200.png,에피나레정
4,K-038884/K-038884_0_0_1_1_75_160_200.png,에피나레정
...,...,...
9607,K-039147/K-039147_0_0_0_1_70_260_200.png,듀카브정30/10밀리그램
9608,K-039147/K-039147_0_1_0_2_75_120_200.png,듀카브정30/10밀리그램
9609,K-039147/K-039147_0_1_1_1_90_260_200.png,듀카브정30/10밀리그램
9610,K-039147/K-039147_0_0_1_0_60_340_200.png,듀카브정30/10밀리그램


In [102]:
spac = df2[df2['Medicine_Name'].str.contains('듀카브정30/10밀리그램|듀카브정30/5밀리그램|루키오정10밀리그램')].index
df4 = df2.drop(spac)
df4 = df4.sample(frac=1).reset_index(drop=True) 
df4

,PNG,Medicine_Name
0,K-038884/K-038884_0_0_1_0_90_320_200.png,에피나레정
1,K-038954/K-038954_0_2_1_0_90_140_200.png,뮤코원캡슐(에르도스테인)
2,K-038910/K-038910_0_0_1_0_70_220_200.png,바실리포미스캡슐
3,K-038958/K-038958_0_2_0_2_70_020_200.png,엘도민캡슐 300mg
4,K-038958/K-038958_0_2_1_2_75_160_200.png,엘도민캡슐 300mg
...,...,...
5719,K-039021/K-039021_0_2_0_2_75_320_200.png,뉴에르도테캡슐
5720,K-038910/K-038910_0_0_0_1_75_000_200.png,바실리포미스캡슐
5721,K-039021/K-039021_0_2_1_2_90_300_200.png,뉴에르도테캡슐
5722,K-038962/K-038962_0_2_1_2_75_220_200.png,앤도민300프리미엄연질캡슐 300mg/PTP


In [103]:
train_df, test_df = train_test_split(df2, test_size=0.2, random_state=42, stratify=df2['Medicine_Name'])

In [104]:
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df['Medicine_Name'])

In [105]:
print("Train set rows: {}".format(train_df.shape[0]))
print("Test  set rows: {}".format(test_df.shape[0]))
print("Val   set rows: {}".format(val_df.shape[0]))

Train set rows: 6151
Test  set rows: 1923
Val   set rows: 1538


In [106]:
def read_image(file_name):
    image = skimage.io.imread(IMAGE_PATH + file_name)
    image = skimage.transform.resize(image, (IMAGE_WIDTH, IMAGE_HEIGHT), mode='reflect')
    return image[:,:,:IMAGE_CHANNELS]

In [107]:
def categories_encoder(dataset, var='Medicine_Name'):
    X = np.stack(dataset['PNG'].apply(read_image))
    y = pd.get_dummies(dataset[var], drop_first=False)
    return X, y

In [ ]:
X_train, y_train = categories_encoder(train_df)
X_val, y_val = categories_encoder(val_df)
X_test, y_test = categories_encoder(test_df)

In [ ]:
n_class = len(cls)
model = tf.keras.models.Sequential()
model.add(tf.keras.applications.densenet.DenseNet121(
    weights="imagenet", 
    include_top=False, 
    input_shape=(64, 64, 3)
    ))
model.add(tf.keras.layers.Flatten(name='flatten'))
model.add(tf.keras.layers.Dense(1024, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(y_train.columns.size, activation='softmax'))

model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
image_generator = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range=180,
        zoom_range = 0.1, 
        width_shift_range=0.1,
        height_shift_range=0.1, 
        horizontal_flip=True,
        vertical_flip=True)
image_generator.fit(X_train)

In [ ]:
%%time
train_model1  = model1.fit_generator(image_generator.flow(X_train, y_train, batch_size=32),
                        epochs=5,
                        validation_data=[X_val, y_val],
                        steps_per_epoch=len(X_train)/32)

In [ ]:
%%time
history = model.fit(X_train, y_train, epochs=4,
                            validation_data=(X_val, y_val))

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])